In [11]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet50
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import numpy as np
# Define data loaders
from torch.utils.data import DataLoader, random_split

# Define preprocessing transformations
import os

In [2]:
# Define the root directory for data
data_root = './data'

# Define preprocessing transformations
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert image to PyTorch tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize pixel values to the range [-1, 1]
])

# Check if MNIST dataset is already downloaded, if not, download it
if not os.path.exists(os.path.join(data_root, 'MNIST')):
    mnist_trainset = torchvision.datasets.MNIST(root=data_root, train=True, download=True, transform=transform)
    mnist_testset = torchvision.datasets.MNIST(root=data_root, train=False, download=True, transform=transform)
else:
    mnist_trainset = torchvision.datasets.MNIST(root=data_root, train=True, download=False, transform=transform)
    mnist_testset = torchvision.datasets.MNIST(root=data_root, train=False, download=False, transform=transform)

# Check if CIFAR-10 dataset is already downloaded, if not, download it
if not os.path.exists(os.path.join(data_root, 'CIFAR10')):
    cifar_trainset = torchvision.datasets.CIFAR10(root=data_root, train=True, download=True, transform=transform)
    cifar_testset = torchvision.datasets.CIFAR10(root=data_root, train=False, download=True, transform=transform)
else:
    cifar_trainset = torchvision.datasets.CIFAR10(root=data_root, train=True, download=False, transform=transform)
    cifar_testset = torchvision.datasets.CIFAR10(root=data_root, train=False, download=False, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
class MNIST_CNN(nn.Module):
    def __init__(self):
        super(MNIST_CNN, self).__init__()
        # Define convolutional layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)  # 1 input channel, 32 output channels, 3x3 kernel
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3)  # 32 input channels, 32 output channels, 3x3 kernel
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3)  # 32 input channels, 64 output channels, 3x3 kernel
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3)  # 64 input channels, 64 output channels, 3x3 kernel
        
        # Define max pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # 2x2 max pooling with stride 2
        
        # Define global average pooling layer
        self.global_pool = nn.AdaptiveAvgPool2d(1)  # Global Average Pooling
        
        # Define fully connected layer
        self.fc = nn.Linear(64, 10)  # 64 input features, 10 output classes

    def forward(self, x):
        # Apply convolution and ReLU activation
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.relu(self.conv2(x))
        # Apply max pooling
        x = self.pool(x)
        x = nn.functional.relu(self.conv3(x))
        x = nn.functional.relu(self.conv4(x))
        # Apply global average pooling
        x = self.global_pool(x)
        # Flatten for fully connected layer
        x = x.view(-1, 64)
        # Fully connected layer
        x = self.fc(x)
        return x

In [4]:
# ResNet50 architecture for CIFAR-10
class CIFAR_ResNet(nn.Module):
    def __init__(self):
        super(CIFAR_ResNet, self).__init__()
        self.resnet50 = resnet50(num_classes=10)

    def forward(self, x):
        return self.resnet50(x)

In [12]:
# Initialize models and optimizer
mnist_model = MNIST_CNN()
cifar_model = CIFAR_ResNet()
mnist_optimizer = optim.Adam(mnist_model.parameters(), lr=1e-3, betas=(0.9, 0.999))
cifar_optimizer = optim.Adam(cifar_model.parameters(), lr=1e-3, betas=(0.9, 0.999))
criterion = nn.CrossEntropyLoss() # multiclassification

# Define the size of the validation set (10% of the training set)
validation_size_mnist = int(0.1 * len(mnist_trainset))
# Split the training set into training and validation sets
mnist_trainset, mnist_valset = random_split(mnist_trainset, [len(mnist_trainset) - validation_size_mnist, validation_size_mnist])

# Define data loaders for training, validation, and testing
mnist_trainloader = DataLoader(mnist_trainset, batch_size=64, shuffle=True)
mnist_valloader = DataLoader(mnist_valset, batch_size=64, shuffle=False)  # No need to shuffle the validation set
mnist_testloader = DataLoader(mnist_testset, batch_size=64, shuffle=False)

# Define the size of the validation set (10% of the training set)
validation_size_cifar = int(0.1 * len(cifar_trainset))

# Split the training set into training and validation sets
cifar_trainset, cifar_valset = random_split(cifar_trainset, [len(cifar_trainset) - validation_size_cifar, validation_size_cifar])

# Define data loaders for training, validation, and testing
cifar_trainloader = DataLoader(cifar_trainset, batch_size=64, shuffle=True)
cifar_valloader = DataLoader(cifar_valset, batch_size=64, shuffle=False)  # No need to shuffle the validation set
cifar_testloader = DataLoader(cifar_testset, batch_size=64, shuffle=False)

In [13]:
def train_with_early_stopping(trainloader, valloader, testloader, model, optimizer, criterion, patience=5):
    best_val_accuracy = 0
    no_improvement_counter = 0

    for epoch in range(100):  # Maximum of 100 epochs
        # Train the model
        model.train()
        for inputs, labels in trainloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # Validate the model
        model.eval()
        val_predictions = []
        val_true_labels = []

        with torch.no_grad():
            for inputs, labels in valloader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1) # torch.max returns both the maximum values and their indices
                val_predictions.extend(predicted.numpy())
                val_true_labels.extend(labels.numpy())

        # Calculate validation accuracy
        val_accuracy = accuracy_score(val_true_labels, val_predictions)
        print(f"Epoch {epoch + 1}: Validation Accuracy: {val_accuracy}")

        # Check for improvement in validation accuracy
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            no_improvement_counter = 0
        else:
            no_improvement_counter += 1
            if no_improvement_counter >= patience:
                print(f"No improvement for {patience} epochs. Early stopping...")
                break

    # Evaluate on the test set after training
    model.eval()
    test_predictions = []
    test_true_labels = []

    with torch.no_grad():
        for inputs, labels in testloader: # batch wise for loop. 
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            test_predictions.extend(predicted.numpy())
            test_true_labels.extend(labels.numpy())

    # Calculate accuracy on the test set
    test_accuracy = accuracy_score(test_true_labels, test_predictions)
    print(f"Test Accuracy: {test_accuracy}")

    return test_accuracy


In [16]:
def create_failure_set(testloader, model):
    
    # Evaluate on the test set after training
    model.eval()
    test_predictions = []
    test_true_labels = []
    failure_set = []

    with torch.no_grad():
        for inputs, labels in testloader: # batch wise for loop. 
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            misclassified_mask = predicted != labels
            failure_set.extend(inputs[misclassified_mask])

    # Calculate accuracy on the test set
    # test_accuracy = accuracy_score(test_true_labels, test_predictions)
    # print(f"Test Accuracy: {test_accuracy}")
    print("The size of the failure set is ", len(failure_set))
    return failure_set

In [14]:
mnist_test_accuracy = train_with_early_stopping(mnist_trainloader, mnist_valloader, mnist_testloader, mnist_model, mnist_optimizer, criterion)


Epoch 1: Validation Accuracy: 0.9368333333333333
Epoch 2: Validation Accuracy: 0.9611666666666666
Epoch 3: Validation Accuracy: 0.9693333333333334
Epoch 4: Validation Accuracy: 0.979
Epoch 5: Validation Accuracy: 0.9818333333333333
Epoch 6: Validation Accuracy: 0.985
Epoch 7: Validation Accuracy: 0.9836666666666667
Epoch 8: Validation Accuracy: 0.9885
Epoch 9: Validation Accuracy: 0.9888333333333333
Epoch 10: Validation Accuracy: 0.9896666666666667
Epoch 11: Validation Accuracy: 0.9891666666666666
Epoch 12: Validation Accuracy: 0.9905
Epoch 13: Validation Accuracy: 0.9915
Epoch 14: Validation Accuracy: 0.9921666666666666
Epoch 15: Validation Accuracy: 0.993
Epoch 16: Validation Accuracy: 0.9915
Epoch 17: Validation Accuracy: 0.9918333333333333
Epoch 18: Validation Accuracy: 0.99
Epoch 19: Validation Accuracy: 0.9905
Epoch 20: Validation Accuracy: 0.9911666666666666
No improvement for 5 epochs. Early stopping...
Test Accuracy: 0.9896


In [19]:
cifar_test_accuracy = train_with_early_stopping(cifar_trainloader, cifar_valloader, cifar_testloader, cifar_model, cifar_optimizer, criterion)

Epoch 1: Validation Accuracy: 0.4446
Epoch 2: Validation Accuracy: 0.4184


In [18]:
F_mnist= create_failure_set(mnist_testloader, mnist_model)

Test Accuracy: nan
The size of the failure set is  104


/Users/tejasr/opt/anaconda3/envs/pytorch/lib/python3.9/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Users/tejasr/opt/anaconda3/envs/pytorch/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
torch.save(mnist_model.state_dict(), 'mnist_model.pth')
torch.save(cifar_model.state_dict(), 'cifar_model.pth')

Adding label noise to the dataset

In [ ]:
# Assuming you have the confusion matrix defined as confusion_matrix
# Example confusion matrix for MNIST (2x2):
# Initialize a 10x10 confusion matrix with ones on the diagonal
confusion_matrix = np.eye(10)
# Set misclassification probabilities
confusion_matrix[1, 7] = 0.6 # Probability of misclassifying label 1 as label 7
confusion_matrix[1, 7] = 0.4  # Probability of misclassifying label 1 as label 7
confusion_matrix[7, 7] = 0.6  # Probability of misclassifying label 7 as label 1
confusion_matrix[7, 1] = 0.4  # Probability of misclassifying label 7 as label 1
confusion_matrix[6, 6] = 0.7  # Probability of misclassifying label 6 as label 9
confusion_matrix[9, 6] = 0.7  # Probability of misclassifying label 9 as label 6
confusion_matrix[6, 9] = 0.3  # Probability of misclassifying label 6 as label 9
confusion_matrix[9, 6] = 0.3  # Probability of misclassifying label 9 as label 6

print(confusion_matrix)

In [ ]:

def flip_labels(dataloader):
    noisy_labels = []
    for _, label in dataloader:
        # Randomly flip the label according to the confusion matrix
        noisy_label = np.random.choice(np.arange(len(confusion_matrix)), p=confusion_matrix[label])
        noisy_labels.append(noisy_label)
    return noisy_labels

# Apply noise to the training labels
noisy_train_labels = flip_labels(mnist_trainloader)
